<a href="https://colab.research.google.com/github/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/blob/main/Machine_Learning_NLP_PreProcesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO


Los emojis los traduce bien pero no como queremos, nos da igual que nos diga face_with_tears_of_joy, el objetivo es que tears y joy se integren con el texto, eso hay que revisarlo.

No sé hasta que punto nos interesa mantener los hastag




In [1]:
# dataframes library
!pip install pyspellchecker 
!pip install contractions
!pip install nltk --upgrade
!pip install emoji

import pandas as pd
import re
import nltk
import sys
import csv
nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter not necessary

from nltk.tokenize.casual import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

#Correcciones y emoji
import emoji
from textblob import TextBlob
from spellchecker import SpellChecker
import contractions
from multiprocessing import Pool
from collections import Counter

#Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

#Feature selection 
from sklearn.feature_selection import SelectKBest, chi2

#Classification
from sklearn.model_selection import train_test_split

from sklearn import neighbors
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels



     |████████████████████████████████| 1.9MB 7.5MB/s 
     |████████████████████████████████| 317kB 7.6MB/s 
     |████████████████████████████████| 245kB 13.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=8d63cd40215a9fc52b57ce6bfe37871f0b1ce44297969f15e1faf16924d3bd0f
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 1.4MB 6.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=ad6ea87a98b68f005cd6f89f7c55c41378de8c20e0a9c1f85326933d3fecf84a
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [2]:
english_stops = set(stopwords.words('english'))
spell = SpellChecker(distance=1)
spell.word_frequency.load_words(['nigga', 'niggas', 'retweet', 'eggplant',
                                 'oreo' ,'gf', 'lmao', 'emoji', 'skippy',
                                 'piercings', 'gaywad','gay-wad','smh','bruh',
                                 'pornhub', 'gtfo', 'mixtape', 'wtf','lmfao',
                                 'twerking','tf','twerk','cowgirls','stfu',
                                 'fohead', 'thots', 'thot', 'friendzone',
                                 'scally', 'bestfriend', 'cuz', 'motherfucker',
                                 'motherfuckers', 'youtube', 'murda', 'idk'])
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
unknown_arr = []
misspelled_arr = []


In [3]:
raw_dataset_tweets = pd.read_csv('https://raw.githubusercontent.com/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/Pre-processing-clean-up-and-revision/labeled_data.csv?token=ANLXPWN4VUDYSEP2V5O6BM273JTYU', index_col=[0])

# Preprocesing:

Mandatory preprocessing steps
 - Remove unseful data: ! "_ $% & / ( ) = _ ˆ* ¡@
 - Remove all capital letters
 - Correct wrong words: https://norvig.com/spell-correct.html
 - Lemmatize all terms

Optional preprocessing steps:
 - Remove contractions: don’t -> do not
 - Remove repeated words: great great show
 - Replace emoticons, for example,🙂 with “smile”, 😉with “ok”


We just need index and tweet content for this first part, so we drop Crowdflowers users information from the dataset.

In [4]:
dataset_tweets_preprocesing = raw_dataset_tweets.copy()
dataset_tweets_preprocesing.drop("count", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("hate_speech", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("offensive_language", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("neither", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("class", axis = 1, inplace = True)

In [5]:
dataset_tweets_preprocesing

,tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...
25291,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
25292,"you've gone and broke the wrong heart baby, an..."
25294,young buck wanna eat!!.. dat nigguh like I ain...
25295,youu got wild bitches tellin you lies


Seleccionamos sólo unos pocos para trabajar al principio más fácilmente

In [6]:
#@title Texto de título predeterminado
num_tweets = 24783 #@param {type:"slider", min:1, max:24783, step:10}

examples_preprocesing = dataset_tweets_preprocesing.iloc[0:num_tweets,0]
examples_preprocesing.reset_index(inplace = True, drop= True)
print(examples_preprocesing)

lista_tweets = []



0        !!! RT @mayasolovely: As a woman you shouldn't...
1        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
                               ...                        
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779    you've gone and broke the wrong heart baby, an...
24780    young buck wanna eat!!.. dat nigguh like I ain...
24781                youu got wild bitches tellin you lies
24782    ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...
Name: tweet, Length: 24783, dtype: object


### Definimos las funciones que procesarán los tweets


In [7]:

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in english_stops])

def stopwords_tokens(arr):
  text_arr = []
  for token in arr:
    if token not in english_stops:
      text_arr.append(token)
  return text_arr

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def demojizar(arr):
  text_arr = []
  trozos_emoji = []
  for token in arr:
    wordemoji = emoji.demojize(token, delimiters=("", ""))
    trozos_emoji = wordemoji.split('_')
    for word in trozos_emoji:
      text_arr.append(word)
  return text_arr

def textConditions(token):
  if re.match("[`!\"_$%&/()=_ˆ*¡,@:.]+",token): #Signos de puntuacion
    return False
  if re.match("[-><~]+",token): #Signos de puntuacion
    return False
  if re.match("[^\u0000-\u007F]+",token): #Unicodes
    return False
  return True

def processArray(arr):
  text_arr = []
  arr = list(set(arr))
  for token in arr:
    if textConditions(token):
      token = stemmer.stem(token)
      token = lemmatizer.lemmatize(token) # < ---- LEMMA
      text_arr.append(token)
  # try: #Aqui iria el tagger de palabras, pero solo me lo hace para letras (ERROR)
  #   arr_tags = nltk.pos_tag(text_arr)
  # except:
  #   arr_tags = text_arr
  return text_arr

def correccion_soft(arr):

  correct_arr = []

  correct = spell.known(arr)
  misspelled = spell.unknown(arr)

  for token in correct:
    correct_arr.append(token)

  for token in misspelled:
    token = replaceThreeOrMore(token)
    candidate = spell.correction(token)
    correct_arr.append(candidate)
  return correct_arr

def replaceThreeOrMore(text):
  pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
  return pattern.sub(r"\1\1", text)

def correccion_hard(arr):

  correct_arr = []

  correct = spell.known(arr)
  misspelled = spell.unknown(arr)

  for token in correct:
    correct_arr.append(token)

  for token in misspelled:
      misspelled_arr.append(token)
      token = replaceThreeOrMore(token)
      candidate = spell.correction(token)
      if candidate == token:
        unknown_arr.append(token)
      else:
        correct_arr.append(candidate)

  return correct_arr



In [8]:
ds_preproc_hard = examples_preprocesing.copy()
ds_preproc_hard = ds_preproc_hard.str.lower() #lower case
ds_preproc_hard = ds_preproc_hard.apply(contractions.fix) #repair contractions

ds_preproc_hard = ds_preproc_hard.str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+[A-Za-z0-9-_]+)','') #Remove users
ds_preproc_hard = ds_preproc_hard.str.replace('\”\“\’[`!\"_$%/()=_ˆ*¡,@:.]+','') #Remove unseful data
ds_preproc_hard = ds_preproc_hard.str.replace('rt','') #Remove rt's


ds_preproc_hard = ds_preproc_hard.apply(stopwords) #Remove Stopwords
ds_preproc_hard = ds_preproc_hard.apply(remove_urls) #Remove url

ds_preproc_hard = ds_preproc_hard.apply(TweetTokenizer().tokenize) #Tokenizar
ds_preproc_hard = ds_preproc_hard.apply(demojizar) #Traducir emoji a texto

ds_preproc_soft = ds_preproc_hard.copy()

ds_preproc_hard = ds_preproc_hard.apply(correccion_hard)


ds_preproc_hard = ds_preproc_hard.apply(stopwords_tokens)
ds_preproc_hard = ds_preproc_hard.apply(processArray) #Traducir emoji a texto

ds_preproc_soft = ds_preproc_soft.apply(correccion_soft)
ds_preproc_soft = ds_preproc_soft.apply(stopwords_tokens)




In [9]:
unknown_arr = list(set(unknown_arr))
print(unknown_arr)

['8n9mi46cbu', 'httptcoqoxkqbyxhe', '9rsfkwj', "justin's", 'barstool', '7gcsmcylb', '647wuie4', 'assholeoh', 'hoeslol', 'concern-trolling', 'bo-stop', "wayne's", 'aklve', '8jdic3wvg', 'httptcoqgnhildttv', '7rujtkei1', '#smalltits', 'httptcotwdkckktaz', 'httptcoaj', 'httptcoflde', 'shityou', '#fallontonight', 'mufuckas', '5yhjtbcnr', 'httptcoqztxjv', '8baxoypgg', 'httptcoucghry', '0lznmjklp', '9zwr3o33iu', '#aamu', 'backatown', '1bc7mxs', '#impeachobama', 'httptcolrydtd', '#yok', '8qnvlxu', '#thegoodlife', 'httptcoyjhuba', 'ajumma', '#returnthefavor', 'mofos', 'sub-tweet', 'stankiest', '#lordwillin', '1sdhby8xt', '#tgdn', '#thingspeoplesay', 'httptcogw', '5auklo', '2qgswipdc', 'winn', 'httptcobmrzfwstva', 'httpstconluzgqv', '1meniz4urw', '42q', 'httptcogyyeayzrjy', 'droppersdick', 'gov-elect', 'timefucking', 'coincky', '198vnt', 'httptcoyctt', '#nbombs', 'linkiden', 'httptcocgt', "hetero's", 'httptcopflatel', 'httptcoufcajnnqtz', '#thewalkingdead', '#ohgawd', 'httptcotbgyikpv', ';}', 'h

In [16]:
Counter(unknown_arr)

Counter({'8n9mi46cbu': 1,
         'httptcoqoxkqbyxhe': 1,
         '9rsfkwj': 1,
         "justin's": 1,
         'barstool': 1,
         '7gcsmcylb': 1,
         '647wuie4': 1,
         'assholeoh': 1,
         'hoeslol': 1,
         'concern-trolling': 1,
         'bo-stop': 1,
         "wayne's": 1,
         'aklve': 1,
         '8jdic3wvg': 1,
         'httptcoqgnhildttv': 1,
         '7rujtkei1': 1,
         '#smalltits': 1,
         'httptcotwdkckktaz': 1,
         'httptcoaj': 1,
         'httptcoflde': 1,
         'shityou': 1,
         '#fallontonight': 1,
         'mufuckas': 1,
         '5yhjtbcnr': 1,
         'httptcoqztxjv': 1,
         '8baxoypgg': 1,
         'httptcoucghry': 1,
         '0lznmjklp': 1,
         '9zwr3o33iu': 1,
         '#aamu': 1,
         'backatown': 1,
         '1bc7mxs': 1,
         '#impeachobama': 1,
         'httptcolrydtd': 1,
         '#yok': 1,
         '8qnvlxu': 1,
         '#thegoodlife': 1,
         'httptcoyjhuba': 1,
         'ajumma'

In [10]:
mispelled_arr = list(set(misspelled_arr))
print(misspelled_arr)

['coldtyga', 'dats', 'cuffin', 'dwn', '1st', '\ue011', 'meclaim', '”', '“', 'keeks', 'murda', 'plz', '2wq5l4q', 'httptcocou', 'txt', '#shots', '#2mw', '3rc', 'httptcobmdsvmc', '#sevenone', '3kf7u', 'httptcobzrdl', '#happyhumpday', '8jsymib5b', 'httptcor', '”', '“', 'teanna', 'tacs', 'wouldn', '’', 'httptcovxbjg', '26gsz', 'simplyaddictedtoguys', '1jl4hi8zmf', 'httptco', 'kanye', 'httptco', '3gzupfumev', 'lemmie', '68nc1', 'httptcotprevwfq', '0s', 'httptcopxmga', 'niggah', 'httptcoi', '8egmnkaj9', 'see-no-evil', 'cuain', '#ahmesehwetness', '#eaglesnation', '#eagles', '6hmt7', 'httptcovbczp', 'soooo', 'httptco', 'awwwwe', '7kpwadlf0r', 'pussyno', '30chance', 'mvps', '9y5u', 'httptcoluubgl', 'ghettohood', 'funnnnnnn', 'httptcozwnbhpdz', '8e', 'tattoostattoo', 'cuteeeee', 'worshiping', '37cejcjou', 'httptcor', '6guyouq6', 'buttcheek', 'httptcoyn', 'tweets', 'lulwhore', 'httptco', '06e77bgwbx', 'beaner', 'alsarabsss', 'nigs', 'bihday', '5vawij1', 'httpstcorkk', '9h', 'httptcoosipbkpr', '#ea

In [15]:
Counter(misspelled_arr)

Counter({'coldtyga': 1,
         'dats': 23,
         'cuffin': 10,
         'dwn': 2,
         '1st': 37,
         '\ue011': 1,
         'meclaim': 1,
         '”': 1530,
         '“': 1564,
         'keeks': 1,
         'murda': 5,
         'plz': 11,
         '2wq5l4q': 1,
         'httptcocou': 1,
         'txt': 4,
         '#shots': 1,
         '#2mw': 1,
         '3rc': 1,
         'httptcobmdsvmc': 1,
         '#sevenone': 1,
         '3kf7u': 1,
         'httptcobzrdl': 1,
         '#happyhumpday': 1,
         '8jsymib5b': 1,
         'httptcor': 14,
         'teanna': 1,
         'tacs': 1,
         'wouldn': 3,
         '’': 218,
         'httptcovxbjg': 1,
         '26gsz': 1,
         'simplyaddictedtoguys': 1,
         '1jl4hi8zmf': 1,
         'httptco': 494,
         'kanye': 21,
         '3gzupfumev': 1,
         'lemmie': 1,
         '68nc1': 1,
         'httptcotprevwfq': 1,
         '0s': 1,
         'httptcopxmga': 1,
         'niggah': 256,
         'httptcoi': 13

In [11]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ds_preproc_hard.to_csv(r'/content/gdrive/My Drive/preprocesado_hard.csv')
ds_preproc_soft.to_csv(r'/content/gdrive/My Drive/preprocesado_soft.csv')

Mounted at /content/gdrive


In [12]:
def identity_tokenizer(text):
    return text

In [13]:
%%time

# TF-IDF
vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer, max_features=8000, lowercase=False)    
vectors = vectorizer.fit_transform(ds_preproc_hard)

feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
tweets_vectorization = pd.DataFrame(denselist, columns=feature_names)


CPU times: user 47 s, sys: 5.67 s, total: 52.7 s
Wall time: 52.7 s


In [14]:
tweets_vectorization

,a lot,a',a-road,a1,a3,a55,a66,aa,aah,aaron,ab,abbey,abbi,abc,abdela,abduct,abdullah,abe,abel,aber,aberdeen,abil,abl,abo,abod,abou,abound,abraham,absent,absolut,absolv,abstract,absurd,abu,abund,abus,abut,ac,aca,academ,...,yuan,yuck,yuh,yummi,yun,yung,yup,yuppi,z,zach,zack,zak,zani,zappa,zara,ze,zebra,zee,zeitgeist,zelah,zelda,zemin,zero,zhou,zim,zimmerman,zine,zing,zion,zip,zipper,zn,zo,zodiac,zoe,zog,zombi,zone,zoo,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
